In [ ]:
import pandas as pd
#2019 Bosch Datasets
bindo2019 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Indonesia/2019/Cleaned/indo2019_Bosch_TRANSLATED_CLEANED.csv")
bmsia2019 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Malaysia/2019/Cleaned/msia2019_Bosch_TRANSLATED_CLEANED.csv")
bphi2019 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Philippines/2019/Cleaned/phi2019_Bosch_TRANSLATED_CLEANED.csv")
bsin2019 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Singapore/2019/Cleaned/sin2019_Bosch_CLEANED.csv")
bthai2019 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Thailand/2019/Cleaned/thai2019_Bosch_TRANSLATED_CLEANED.csv")
bviet2019 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Vietnam/2019/Cleaned/viet2019_Bosch_TRANSLATED_CLEANED.csv")

#2020 Bosch Datasets
bindo2020 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Indonesia/2020/Cleaned/indo2020_Bosch_TRANSLATED_CLEANED.csv")
bmsia2020 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Malaysia/2020/Cleaned/msia2020_Bosch_TRANSLATED_CLEANED.csv")
bphi2020 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Philippines/2020/Cleaned/phi2020_Bosch_TRANSLATED_CLEANED.csv")
bsin2020 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Singapore/2020/Cleaned/sin2020_Bosch_CLEANED.csv")
bthai2020 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Thailand/2020/Cleaned/thai2020_Bosch_TRANSLATED_CLEANED.csv")
bviet2020 = pd.read_csv("/content/drive/MyDrive/Bosch/Binned/Vietnam/2020/Cleaned/viet2020_Bosch_TRANSLATED_CLEANED.csv")

#2019 datasets
b2019= bindo2019.append([bmsia2019, bphi2019, bsin2019, bthai2019, bviet2019],ignore_index = True)
b2020= bindo2020.append([bmsia2020, bphi2020, bsin2020, bthai2020, bviet2020],ignore_index = True)
wipersexcel= pd.read_excel("/content/drive/MyDrive/Bosch/Wipers__Asean.xlsx")

In [ ]:
#def split_subcat(file):
countrylist= ['msia', 'phi','sin','thai', 'viet']
for i in range(len(blist2019)):
  file= blist2019[i]
  country= countrylist[i]
  for j in range(len(file)):
    new = file["Relevant Nr prod key text"].str.split(",", n = 1, expand = True) 
    file["Relevant Nr prod key text1"]= new[0]
    file["Relevant Nr prod key text2"]= new[1]

In [ ]:
def subcat(file, country, year):
    g= file.groupby('Relevant Nr prod key text1')
    g.ngroups
    g.apply(lambda x: x.to_csv(r'/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/wipers_bosch_'+country+year+'_{}.csv'.format(x.name), index=False))

In [ ]:
#bmsia2019, bphi2019, bsin2019, bthai2019, bviet2019
#bmsia2020, bphi2020, bsin2020, bthai2020, bviet2020
subcat(bmsia2019, "msia", "2019")
subcat(bsin2019, "sin", "2019")
subcat(bphi2019, "phi", "2019")
#subcat(bthai2019, "thai", "2019")
#subcat(bviet2019, "viet", "2019")
subcat(bmsia2020, "msia", "2020")
subcat(bsin2020, "sin", "2020")
subcat(bphi2020, "phi", "2020")
#subcat(bthai2020, "thai", "2020")
#subcat(bviet2020, "viet", "2020")

In [ ]:
import nltk
from string import punctuation
punctuation_symbols = []
for symbol in punctuation:
    punctuation_symbols.append((symbol, ''))

In [ ]:
#remove punctuation from reviewText
import string
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

# Remove Digits
def remove_digits(x):
    x = ''.join([i for i in x if not i.isdigit()])
    return x

# Remove Stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

def remove_stop_words(x):
    x = ' '.join([i for i in x.lower().split(' ') if i not in stop])
    return x

# Change to LowerCase Words
def to_lower(x):
    return x.lower()

#changing word to root word
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
import numpy as np
def tfidf(file):
  file= file.dropna(subset= ['reviewText_Translated']).reset_index(drop=True)
  review= file['reviewText_Translated']
  review= review.apply(remove_digits)
  review= review.apply(remove_punctuation)
  review= review.apply(remove_stop_words)
  review= review.apply(to_lower)
  # Apply TFIDF to "item_description", 
  tv = TfidfVectorizer(max_features=55000, ngram_range=(1, 2), stop_words='english')
  X_description = tv.fit_transform(review)
  feature_array = np.array(tv.get_feature_names())
  tv_sorting= np.argsort(X_description.toarray()).flatten()[::-1]
  # sum tfidf frequency of each term through documents
  sums = X_description.sum(axis=0)
  terms = tv.get_feature_names()
  # connecting term to its sums frequency
  data = []
  sub= file['Relevant Nr prod key text1'][0]
  for col, term in enumerate(terms):
      data.append( (term, sums[0,col], sub))

  ranking = pd.DataFrame(data, columns=['word/phrase','rank', 'sub'])
  ranking= ranking.sort_values('rank', ascending=False).reset_index(drop=True)
  #ranking= ranking.reset_index(drop=True)
  #print(ranking.sort_values('rank', ascending=False).head(20))
  #df= pd.DataFrame(columns= ['word/phrase'])
  #if (len(tv_sorting)>20):
  #  for x in range (21):
  #    index= tv_sorting[x]
  #    word= feature_array[index]
  #    sub= file['Relevant Nr prod key text1'][0]
  #    df = df.append({'word/phrase': word,
  #                    'subcat': sub}, ignore_index=True)
    #print(df)
  return (ranking)

In [ ]:
#for Singapore 2019 Bosch Subcat
import os
directory = '/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Singapore/2019'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      print(filename)
      file= pd.read_csv("/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Singapore/2019/"+filename)
      file['reviewText_Translated']= file['reviewText']
      df= tfidf(file)
      #print (df)
      if (df is not None and len(df)>20):
        #print(df)
        name= "/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Singapore/2019/USP/Raw/usp_"+filename
        name= str(name)
        #print(name)
        df= df.head(20)
        df.to_csv(name, index=False)

print('done')

#for Singapore 2020 Bosch Subcat
import os
directory = '/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Singapore/2020'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      print(filename)
      file= pd.read_csv("/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Singapore/2020/"+filename)
      file['reviewText_Translated']= file['reviewText']
      df= tfidf(file)
      #print (df)
      if (df is not None and len(df)>20):
        #print(df)
        name= "/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Singapore/2020/USP/Raw/usp_"+filename
        name= str(name)
        #print(name)
        df= df.head(20)
        df.to_csv(name, index=False)

print('done')

In [ ]:
#for Philippines 2019 Bosch Subcat
directory = '/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Philippines/2019'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      print(filename)
      file= pd.read_csv("/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Philippines/2019/"+filename)
      df= tfidf(file)
      #print (df)
      if (df is not None and len(df)>20):
        #print(df)
        name= "/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Philippines/2019/USP/Raw/usp_"+filename
        name= str(name)
        #print(name)
        df= df.head(20)
        df.to_csv(name, index=False)

print('done')

#for Philippines 2020 Bosch Subcat
directory = '/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Philippines/2020'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      print(filename)
      file= pd.read_csv("/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Philippines/2020/"+filename)
      df= tfidf(file)
      #print (df)
      if (df is not None and len(df)>20):
        #print(df)
        name= "/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Philippines/2020/USP/Raw/usp_"+filename
        name= str(name)
        #print(name)
        df= df.head(20)
        df.to_csv(name, index=False)

print('done')

In [ ]:
#for Malaysia 2019 Bosch Subcat
directory = '/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Malaysia/2019'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      print(filename)
      file= pd.read_csv("/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Malaysia/2019/"+filename)
      df= tfidf(file)
      #print (df)
      if (df is not None and len(df)>20):
        #print(df)
        name= "/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Malaysia/2019/USP/Raw/usp_"+filename
        name= str(name)
        #print(name)
        df= df.head(20)
        df.to_csv(name, index=False)

print('done')

#for Malaysia 2020 Bosch Subcat
directory = '/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Malaysia/2020'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      print(filename)
      file= pd.read_csv("/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Malaysia/2020/"+filename)
      df= tfidf(file)
      #print (df)
      if (df is not None and len(df)>20):
        #print(df)
        name= "/content/drive/MyDrive/Bosch/Sentiment Analysis stuff/Data/Wipers Subcat/Malaysia/2020/USP/Raw/usp_"+filename
        name= str(name)
        #print(name)
        df= df.head(20)
        df.to_csv(name, index=False)

print('done')